Архив вопросов

http://gameshows.ru/wiki/%D0%9A%D0%B0%D1%82%D0%B5%D0%B3%D0%BE%D1%80%D0%B8%D1%8F:%D0%A1%D0%B2%D0%BE%D1%8F_%D0%B8%D0%B3%D1%80%D0%B0

Вопросы игры от 21.01.2017 г.

http://gameshows.ru/wiki/%D0%A1%D0%B2%D0%BE%D1%8F_%D0%B8%D0%B3%D1%80%D0%B0_(%D0%9E%D0%B1%D0%B7%D0%BE%D1%80_%D0%B2%D1%8B%D0%BF%D1%83%D1%81%D0%BA%D0%B0_2017-01-21)

In [1]:
# импорт библиотек
import sqlite3
import psycopg2
import requests
from requests_html import AsyncHTMLSession
from bs4 import BeautifulSoup
import os
import requests
import random
import json
from dotenv import load_dotenv
load_dotenv()

True

In [2]:
# переменные окружения
TOKEN = os.environ.get("TOKEN")
GROUP_ID = os.environ.get("GROUP_ID")

In [2]:
# адрес страницы с вопросами игры
url = "http://gameshows.ru/wiki/%D0%A1%D0%B2%D0%BE%D1%8F_%D0%B8%D0%B3%D1%80%D0%B0_(%D0%9E%D0%B1%D0%B7%D0%BE%D1%80_%D0%B2%D1%8B%D0%BF%D1%83%D1%81%D0%BA%D0%B0_2017-01-21)"

In [3]:
async def get_website(url: str) -> bytes:

    """ функция получения содержания страницы
    с вопросами игры

    Args:
        url (str): адрес страницы с вопросами игры

    Returns:
        bytes: содержание страницы
    """

    asession = AsyncHTMLSession() 

    r = await asession.get(url)

    await r.html.arender(sleep = 10) # sleeping is optional but do it just in case

    html = r.html.raw_html # this can be returned as your result

    await asession.close() # this part is important otherwise the Unwanted Kill.Chrome Error can Occur 

    return html

In [4]:
# получение содержания страницы
response = await get_website(url)

In [6]:
# запись содержимого в файл
with open("raw_data/questions_html.txt", "wb") as f:
        f.write(response)

In [5]:
# считывание файла
with open("raw_data/questions_html.txt", "rb") as f:
        response = f.read()

In [6]:
# парсинг данных
soup = BeautifulSoup(response, "html.parser")

In [8]:
# тема вопроса
soup.find_all("h4")[0].text.split(" (")[0]

'Семья композитора'

In [69]:
# количество очков (баллов)
soup.find_all("h4")[0].text.split(" (")[-1][:-1]

'300'

In [10]:
# вопрос
soup.find("div", {"class" : "mw-content-ltr"}).find_all("p")[9].text.strip()

'На вопрос, как им удавалось сохранить семейное счастье, эти супруги отвечали: «Зная мы ответ, давно бы получили Нобелевскую премию по психологии».'

In [11]:
# ответ
soup.find("div", {"class" : "mw-content-ltr"}).find_all("p")[10].text.split(":")[-1].strip()

'Майя Плисецкая и Родион Щедрин'

In [12]:
# вопрос
soup.find("div", {"class" : "mw-content-ltr"}).find_all("p")[11].text.strip()

'В стихотворении «Разбойник» 1915 года он писал:\n«Стухнут звёзды, стухнет месяц,\nСтихнет песня соловья,\nВ чернобылье перелесиц\nС кистенём засяду я».'

In [13]:
# ответ
soup.find("div", {"class" : "mw-content-ltr"}).find_all("p")[12].text.split(":")[-1].strip()

'Сергей Александрович Есенин'

In [88]:
# наименования файла фото вопроса (http://gameshows.ru/w/images/c/cf/RU-SI-2017-01-21-1.jpg)
soup.find_all("a", {"class" : "image"})[5].get("href").split(":")[-1]

'RU-SI-2017-01-21-5.jpg'

In [78]:
soup.find_all("img", {"class" : "thumbimage"})[7].get("src").split("/200px")[0]

'/w/images/thumb/e/e4/RU-SI-2017-01-21-7.jpg'

In [14]:
# тема вопроса с фото
soup.find_all("div", {"class" : "thumbcaption"})[5].text.split(" (")[0]

'Семья композитора'

In [16]:
# количество очков (баллов)
soup.find_all("div", {"class" : "thumbcaption"})[5].text.split(" (")[1][:-1]

'100'

### Создание базы данных

In [3]:
# подключение к базе данных (в случае отсутствия база данных создается)
connection_dev = sqlite3.connect("data/questions.db")
# создание объекта подключения
cursor_dev = connection_dev.cursor()
print("База данных создана и успешно подключена к SQLite")

База данных создана и успешно подключена к SQLite


In [20]:
# закрытие соединения с базой данных
#connection.close()

#### Создание таблицы "Темы"
* theme_id - идентификатор темы
* theme_title - название темы

In [21]:
# создание таблицы themes
cursor_dev.execute("""CREATE TABLE IF NOT EXISTS themes(
                  theme_id INTEGER PRIMARY KEY,
                  theme_title TEXT UNIQUE NOT NULL);""")
connection_dev.commit()

#### Создание таблицы "Впоросы"
* question_id - идентификатор вопроса
* question_title - содержание вопроса
* answer - ответ на вопрос
* points - количество баллов (очков)
* image_href - ссылка на фото вопроса
* audio_href - ссылка на аудио вопроса
* theme_id - идентифиатор темы

In [155]:
# создание таблицы questions
cursor_dev.execute("""CREATE TABLE IF NOT EXISTS questions(
                  question_id INTEGER PRIMARY KEY,
                  question_title TEXT UNIQUE NOT NULL,
                  answer TEXT NOT NULL,
                  points INTEGER NOT NULL,
                  image_href TEXT,
                  audio_href TEXT,
                  theme_id INTEGER NOT NULL);""")
connection_dev.commit()

In [154]:
# удаление таблицы
# cursor_dev.execute("DROP TABLE questions")
# connection_dev.commit()

In [114]:
# темы
themes = [theme.text.split(" (")[0] for theme in soup.find_all("h4")[:-1] if theme.text.split(" (")[0] != "Итог раунда"]
themes[:6]

['Семья композитора',
 'Книжные разбойники',
 'Семья композитора',
 'Семья композитора',
 'Семья композитора',
 'Меткий стрелок']

In [115]:
# проверка данных
len(themes) 

90

In [116]:
# уникальные темы
themes_unique = list(set(themes))
themes_unique

['Книжные разбойники',
 'Транспорт',
 'Маленькие хищники',
 'Параллели и меридианы',
 'Станки',
 'На помощь!',
 '…ура…',
 'Чашка чая',
 'Кинофразы',
 'Зоопарки',
 'Президенты',
 'Меткий стрелок',
 'Бароны',
 'Вопросы от…',
 'Ошибочка вышла',
 'Калмыкия',
 'Вот так номер!',
 'Семья композитора']

In [117]:
# преобразование для записи в базу данных
themes_unique = [(theme, ) for theme in themes_unique]
themes_unique

[('Книжные разбойники',),
 ('Транспорт',),
 ('Маленькие хищники',),
 ('Параллели и меридианы',),
 ('Станки',),
 ('На помощь!',),
 ('…ура…',),
 ('Чашка чая',),
 ('Кинофразы',),
 ('Зоопарки',),
 ('Президенты',),
 ('Меткий стрелок',),
 ('Бароны',),
 ('Вопросы от…',),
 ('Ошибочка вышла',),
 ('Калмыкия',),
 ('Вот так номер!',),
 ('Семья композитора',)]

In [32]:
# запись в базу данных
cursor_dev.executemany("""INSERT INTO themes (theme_title)
                      VALUES(?);""", themes_unique) 
connection_dev.commit()

In [33]:
# проверка внесения записи
cursor_dev.execute("""SELECT COUNT(*)
                  FROM themes;""")
cursor_dev.fetchone()[0], len(themes_unique)

(18, 18)

Данные записаны **успешно**

In [110]:
# получение id наименования темы
def select_theme_id(theme_title: str) -> int:

    """ функция получения идентификатора темы
    по ее наименованию

    Args:
        theme_title (str): наименование темы

    Returns:
        int: идентификатор темы
    """
    try:
        cursor_dev.execute("""SELECT theme_id
                                FROM themes
                                WHERE theme_title == ?;""", (theme_title,))
    
        return cursor_dev.fetchone()[0]
    except:
        print(f"Такой темы нет: {theme_title}")

In [156]:
# тест функции select_theme_id
select_theme_id("Калмыкия")

18

In [157]:
# количество очков (баллов)
points = [int(point.text.split(" (")[-1][:-1].replace(" ", "")) for point in soup.find_all("h4")[:-1] if point.text.split(" (")[-1][:-1] != "Итог раунд"]
points[:6]

[300, 300, 500, 400, 200, 300]

In [158]:
# проверка данных
len(points)

90

In [159]:
# вопросы
question_titles = [title.text.strip() for title in  soup.find("div", {"class" : "mw-content-ltr"}).find_all("p")[9:-7] if "Правильный ответ"  not in title.text and "Темы:" not in title.text]
question_titles[:6]

['На вопрос, как им удавалось сохранить семейное счастье, эти супруги отвечали: «Зная мы ответ, давно бы получили Нобелевскую премию по психологии».',
 'В стихотворении «Разбойник» 1915 года он писал:\r\n«Стухнут звёзды, стухнет месяц,\r\nСтихнет песня соловья,\r\nВ чернобылье перелесиц\r\nС кистенём засяду я».',
 'Работая над «Парсифалем», он, по его словам, не мог сочинить ни ноты, если рядом не было жены Козимы. На день рождения супруги он нанял оркестр, исполнивший прелюдию к «Парсифалю».',
 'Фридрих Вик долго не давал согласия на брак дочери, но по решению суда Клара стала именно его женой и родила ему 8 детей. С отцом помирились года через три.',
 'После поездки в США и Канаду снайпер Людмила Павличенко служила инструктором на курсах комсостава, носивших это говорящее название.',
 'Вопрос-аукцион. Мексиканка Мельба Приа ездит по Дели на трёхколёсном мотороллере; и пробки не страшны и выхлоп невелик. Этот высокий пост даму не смущает.']

In [160]:
question_titles.insert(2, "Его брак с Марии-Анне Келлер не принёс счастья ни одному из супругов. Детей у них не было, а партитуры использовались на папильотки и подставки для паштета.")

In [161]:
question_titles.insert(10, "Национальный большой театр в Пекине поставил эту оперу. Композитор Хао Вэй написал 18-минутное продолжение, добавив к персонажам принцессу Лоу-Линь и китайского Ангела.")

In [162]:
# проверка данных
len(question_titles)

90

In [163]:
# ответ
answers = [answer.text.split(":")[-1].strip() for answer in soup.find("div", {"class" : "mw-content-ltr"}).find_all("p")[9:-3] if "Правильный ответ" in answer.text]
answers[:6]

['Майя Плисецкая и Родион Щедрин',
 'Сергей Александрович Есенин',
 'Рихард Вагнер',
 'Роберт Шуман',
 '«Выстрел»',
 'Посол Мексики в Индии']

In [164]:
answers.insert(2, "Йозеф Гайдн")

In [165]:
answers.insert(10, "Турандот")

In [166]:
# проверка данных
len(answers)

90

In [167]:
# преобразование для записи в базу данных
questions = []
for i in range(90):
    questions.append((question_titles[i], answers[i], points[i], themes[i]))
questions[:4]

[('На вопрос, как им удавалось сохранить семейное счастье, эти супруги отвечали: «Зная мы ответ, давно бы получили Нобелевскую премию по психологии».',
  'Майя Плисецкая и Родион Щедрин',
  300,
  'Семья композитора'),
 ('В стихотворении «Разбойник» 1915 года он писал:\r\n«Стухнут звёзды, стухнет месяц,\r\nСтихнет песня соловья,\r\nВ чернобылье перелесиц\r\nС кистенём засяду я».',
  'Сергей Александрович Есенин',
  300,
  'Книжные разбойники'),
 ('Его брак с Марии-Анне Келлер не принёс счастья ни одному из супругов. Детей у них не было, а партитуры использовались на папильотки и подставки для паштета.',
  'Йозеф Гайдн',
  500,
  'Семья композитора'),
 ('Работая над «Парсифалем», он, по его словам, не мог сочинить ни ноты, если рядом не было жены Козимы. На день рождения супруги он нанял оркестр, исполнивший прелюдию к «Парсифалю».',
  'Рихард Вагнер',
  400,
  'Семья композитора')]

In [168]:
# запись в базу данных
def insetr_questions(question_title: str, answer: str, points: int, theme_title: str) -> None:

    """ функция записи содержания вопроса  в базу данны
    Args:
        question_title (str): содержание вопроса
        answers (str): ответ на вопрос
        point (int): количество баллов (очков)
        theme_title (str): наименование темы
    """

    theme_id = select_theme_id(theme_title)

    cursor_dev.execute("""INSERT INTO questions
                    (question_title, answer, points, theme_id)
                    VALUES(?, ?, ?, ?);""", (question_title, answer, points, theme_id,)) 
    connection_dev.commit()

In [169]:
# запись в базу данных
for question_title, answer, points, theme_title in questions:
    insetr_questions(question_title, answer, points, theme_title)   

In [170]:
# проверка внесения записи
cursor_dev.execute("""SELECT COUNT(*)
                  FROM questions;""")
cursor_dev.fetchone()[0], len(questions)

(90, 90)

In [171]:
# проверка внесения записи
cursor_dev.execute("""SELECT *
                  FROM questions
                  LIMIT 10;""")
cursor_dev.fetchall()

[(1,
  'На вопрос, как им удавалось сохранить семейное счастье, эти супруги отвечали: «Зная мы ответ, давно бы получили Нобелевскую премию по психологии».',
  'Майя Плисецкая и Родион Щедрин',
  300,
  None,
  None,
  14),
 (2,
  'В стихотворении «Разбойник» 1915 года он писал:\r\n«Стухнут звёзды, стухнет месяц,\r\nСтихнет песня соловья,\r\nВ чернобылье перелесиц\r\nС кистенём засяду я».',
  'Сергей Александрович Есенин',
  300,
  None,
  None,
  5),
 (3,
  'Его брак с Марии-Анне Келлер не принёс счастья ни одному из супругов. Детей у них не было, а партитуры использовались на папильотки и подставки для паштета.',
  'Йозеф Гайдн',
  500,
  None,
  None,
  14),
 (4,
  'Работая над «Парсифалем», он, по его словам, не мог сочинить ни ноты, если рядом не было жены Козимы. На день рождения супруги он нанял оркестр, исполнивший прелюдию к «Парсифалю».',
  'Рихард Вагнер',
  400,
  None,
  None,
  14),
 (5,
  'Фридрих Вик долго не давал согласия на брак дочери, но по решению суда Клара стала 

Данные записаны **успешно**

In [172]:
# наименования файла фото вопроса 
image_hrefs = [image_href.get("src").split("/200px")[0] for image_href in soup.find_all("img", {"class" : "thumbimage"})[4:-1]]
image_hrefs[:6]

['/w/images/thumb/f/f9/RU-SI-2017-01-21-4.jpg',
 '/w/images/thumb/7/76/RU-SI-2017-01-21-5.jpg',
 '/w/images/thumb/6/62/RU-SI-2017-01-21-6.jpg',
 '/w/images/thumb/e/e4/RU-SI-2017-01-21-7.jpg',
 '/w/images/thumb/6/60/RU-SI-2017-01-21-8.jpg',
 '/w/images/thumb/5/57/RU-SI-2017-01-21-9.jpg']

In [173]:
# проверка данных
len(image_hrefs)

21

In [174]:
# тема вопроса с фото
theme_title_images = [theme_title_image.text.split(" (")[0] if "Марина Сучкова" not in theme_title_image.text else "Вопросы от…" \
    for theme_title_image in soup.find_all("div", {"class" : "thumbcaption"})[4:-1]]
theme_title_images[:6]

['Семья композитора',
 'Семья композитора',
 'Констанция Вебер',
 'Дом Моцарта в Вене',
 'Вокруг футбола',
 'Друзья']

In [175]:
# проверка данных
len(theme_title_images)

21

In [176]:
theme_title_images[4] = "Транспорт"

In [177]:
theme_title_images[5] = "Книжные разбойники"

In [178]:
theme_title_images[8] = "Президенты"

In [179]:
theme_title_images[19] = "Вот так номер!"

In [180]:
theme_title_images[20] = "Калмыкия"

In [181]:
# количество очков (баллов) вопроса с фото
points_images = []
for points_image in soup.find_all("div", {"class" : "thumbcaption"})[4:-1]:
    if "Марина Сучкова" in points_image.text:
        points_images.append(200)
    elif "(" not in points_image.text:
        points_images.append(100)
    else:
        points_images.append(int(points_image.text.split(" (")[1][:-1].replace(" ", "")))
    
    [points_image.text for points_image in soup.find_all("div", {"class" : "thumbcaption"})[4:-1] if "Марина Сучкова" ]
points_images[:5]

[300, 100, 100, 100, 100]

In [182]:
# проверка данных
len(points_images)

21

In [183]:
# преобразование для записи в базу данных
images = []
for i in range(21):
    images.append((image_hrefs[i].replace("thumb/", ""), theme_title_images[i], points_images[i]))
images[:4]

[('/w/images/f/f9/RU-SI-2017-01-21-4.jpg', 'Семья композитора', 300),
 ('/w/images/7/76/RU-SI-2017-01-21-5.jpg', 'Семья композитора', 100),
 ('/w/images/6/62/RU-SI-2017-01-21-6.jpg', 'Констанция Вебер', 100),
 ('/w/images/e/e4/RU-SI-2017-01-21-7.jpg', 'Дом Моцарта в Вене', 100)]

In [184]:
len(images)

21

In [185]:
images

[('/w/images/f/f9/RU-SI-2017-01-21-4.jpg', 'Семья композитора', 300),
 ('/w/images/7/76/RU-SI-2017-01-21-5.jpg', 'Семья композитора', 100),
 ('/w/images/6/62/RU-SI-2017-01-21-6.jpg', 'Констанция Вебер', 100),
 ('/w/images/e/e4/RU-SI-2017-01-21-7.jpg', 'Дом Моцарта в Вене', 100),
 ('/w/images/6/60/RU-SI-2017-01-21-8.jpg', 'Транспорт', 100),
 ('/w/images/5/57/RU-SI-2017-01-21-9.jpg', 'Книжные разбойники', 500),
 ('/w/images/8/8d/RU-SI-2017-01-21-10.jpg', 'Транспорт', 400),
 ('/w/images/4/43/RU-SI-2017-01-21-11.jpg', 'Вопросы от…', 200),
 ('/w/images/b/bc/RU-SI-2017-01-21-12.jpg', 'Президенты', 1000),
 ('/w/images/7/7f/RU-SI-2017-01-21-13.jpg', 'Президенты', 400),
 ('/w/images/2/22/RU-SI-2017-01-21-14.jpg', 'Вопросы от…', 800),
 ('/w/images/f/fa/RU-SI-2017-01-21-15.jpg', 'Вопросы от…', 1000),
 ('/w/images/1/1b/RU-SI-2017-01-21-16.jpg', 'Ошибочка вышла', 1000),
 ('/w/images/1/19/RU-SI-2017-01-21-17.jpg', 'Ошибочка вышла', 1000),
 ('/w/images/1/10/RU-SI-2017-01-21-18.jpg', 'Ошибочка вышла',

In [143]:
# сохранение файла
def get_image(image_href: str) -> None:

    """функция сохранения фото вопроса

    Args:
        image_href (str): наименования файла фото вопроса 
    """

    url = "http://gameshows.ru"
    r = requests.get(url + image_href)
    image_href = image_href.split("/")[-1]
    # запись содержимого в файл
    with open(f"content/images/{image_href}", 'wb') as f:
        f.write(r.content)


In [186]:
# запись в базу данных
def update_images_questions(image_href: str, theme_title: str, points: int) -> None:

    """функция записи фото вопроса в базу данны

    Args:
        image_href (str): наименования файла фото вопроса
        theme_title (str): наименование темы
        points (int): количество баллов (очков)
    """
    get_image(image_href)

    theme_id = select_theme_id(theme_title)
    cursor_dev.execute("""UPDATE questions
                        SET image_href = ?
                        WHERE theme_id = ? AND points = ?;""",
                        (image_href, theme_id, points,)) 
    connection_dev.commit()

In [187]:
# запись в базу данных
for image_href, theme_title, points in images:
    update_images_questions(image_href, theme_title, points)

Такой темы нет: Констанция Вебер
Такой темы нет: Дом Моцарта в Вене


In [188]:
# проверка внесения записи
cursor_dev.execute("""SELECT COUNT(*)
                  FROM questions
                  WHERE image_href NOT NULL;""")
cursor_dev.fetchall()

[(17,)]

In [189]:
# ссылки на аудио-файл
# http://gameshows.ru/w/images/a/ad/RU-SI-2017-01-21-2.mp3
audio_hrefs = soup.find_all("span", {"class" : "dewWrapper"})
audio_hrefs

[<span class="dewWrapper"> <object data="/w/extensions/Mp3Handler/dewplayer.swf?mp3=/w/images/c/c2/RU-SI-2017-01-21-1.mp3&amp;showtime=1" height="21" style="vertical-align: top;" type="application/x-shockwave-flash" width="197"> <param name="wmode" value="transparent"/> <param name="movie" value="/w/extensions/Mp3Handler/dewplayer.swf?mp3=/w/images/c/c2/RU-SI-2017-01-21-1.mp3&amp;showtime=1"/> </object> <a href="/w/images/c/c2/RU-SI-2017-01-21-1.mp3" title="Download"> <img alt="Download" src="/w/extensions/Mp3Handler/download.png" style="vertical-align: top; margin-left: 0pt; margin-top: 2px;"/> </a> </span>,
 <span class="dewWrapper"> <object data="/w/extensions/Mp3Handler/dewplayer.swf?mp3=/w/images/a/ad/RU-SI-2017-01-21-2.mp3&amp;showtime=1" height="21" style="vertical-align: top;" type="application/x-shockwave-flash" width="197"> <param name="wmode" value="transparent"/> <param name="movie" value="/w/extensions/Mp3Handler/dewplayer.swf?mp3=/w/images/a/ad/RU-SI-2017-01-21-2.mp3&amp;

In [190]:
# наименование файла
audio_hrefs[0].find("object").get("data").split("mp3=")[-1].replace("&showtime=1", "")

'/w/images/c/c2/RU-SI-2017-01-21-1.mp3'

In [191]:
# наименование файла
audio_hrefs = [audio_href.find("object").get("data").split("mp3=")[-1].replace("&showtime=1", "") for audio_href in audio_hrefs]
audio_hrefs

['/w/images/c/c2/RU-SI-2017-01-21-1.mp3',
 '/w/images/a/ad/RU-SI-2017-01-21-2.mp3']

In [162]:
# сохранение файла
def get_audio(audio_href: str) -> None:

    """функция сохранения айдио вопроса

    Args:
        audio_href (str): наименования файла аудио вопроса 
    """

    url = "http://gameshows.ru"
    r = requests.get(url + audio_href)
    audio_href = audio_href.split("/")[-1]
    # запись содержимого в файл
    with open(f"content/audios/{audio_href}", 'wb') as f:
        f.write(r.content)

In [193]:
# запись в базу данных
def update_audio_questions(audio_href: str, theme_title: str, points: int) -> None:

    """функция записи аудио вопроса в базу данны

    Args:
        audio_href (str): наименования файла аудио вопроса
        theme_title (str): наименование темы
        points (int): количество баллов (очков)
    """
    # get_audio(audio_href)

    try:
        theme_id = select_theme_id(theme_title)
        cursor_dev.execute("""UPDATE questions
                        SET audio_href = ?
                        WHERE theme_id = ? AND points = ?;""",
                        (audio_href, theme_id, points,)) 
        connection_dev.commit()
    except:
        pass

In [194]:
theme_title_audio = ["Семья композитора", "…ура…"]
points_audio = [500, 400]

In [195]:
# преобразование для записи в базу данных
audios = []
for i in range(2):
    audios.append((audio_hrefs[i], theme_title_audio[i], points_audio[i]))
audios[:4]

[('/w/images/c/c2/RU-SI-2017-01-21-1.mp3', 'Семья композитора', 500),
 ('/w/images/a/ad/RU-SI-2017-01-21-2.mp3', '…ура…', 400)]

In [196]:
# запись в базу данных
for audio_href, theme_title, points in audios:
    update_audio_questions(audio_href, theme_title, points)

In [197]:
# проверка внесения записи
cursor_dev.execute("""SELECT *
                  FROM questions
                  WHERE audio_href NOT NULL
                  LIMIT 5;""")
cursor_dev.fetchall()

[(3,
  'Его брак с Марии-Анне Келлер не принёс счастья ни одному из супругов. Детей у них не было, а партитуры использовались на папильотки и подставки для паштета.',
  'Йозеф Гайдн',
  500,
  None,
  '/w/images/c/c2/RU-SI-2017-01-21-1.mp3',
  14),
 (11,
  'Национальный большой театр в Пекине поставил эту оперу. Композитор Хао Вэй написал 18-минутное продолжение, добавив к персонажам принцессу Лоу-Линь и китайского Ангела.',
  'Турандот',
  400,
  None,
  '/w/images/a/ad/RU-SI-2017-01-21-2.mp3',
  8)]

In [198]:
# вывод наименования таблиц в базе данных
cursor_dev.execute("""SELECT name
                  FROM sqlite_master
                  WHERE type='table';""")
cursor_dev.fetchall()

[('themes',), ('questions',)]

In [173]:
# закрытие соединения с базой данных
connection_dev.close()

### ETL

In [4]:
# создание подключения к базе данных
connect_prod = psycopg2.connect(database="my_game",
                           user="player",
                           password="my_pass",
                           host="localhost",
                           port="5432")

print('База данных успешно открыта')

База данных успешно открыта


In [5]:
# создание объекта курсора подключения к базе данных
cursor_prod = connect_prod.cursor()

In [6]:
# вывод наименований таблиц в базе данных
cursor_prod.execute("""SELECT table_name
                       FROM information_schema.tables
                       WHERE table_schema NOT IN ('information_schema', 'pg_catalog')
                             AND table_schema IN('public', 'myschema');""")
cursor_prod.fetchall()

[('alembic_version',),
 ('admins',),
 ('games',),
 ('game_sessions',),
 ('players',),
 ('themes',),
 ('questions',)]

In [11]:
# темы игры
cursor_dev.execute("""SELECT theme_title, MIN(points) / 100
                      FROM questions
                      INNER JOIN themes USING(theme_id)
                      GROUP BY theme_title
                      ORDER BY points;""")
themes = cursor_dev.fetchall()
themes

[('Зоопарки', 1),
 ('Книжные разбойники', 1),
 ('Меткий стрелок', 1),
 ('Семья композитора', 1),
 ('Транспорт', 1),
 ('…ура…', 1),
 ('Вопросы от…', 2),
 ('Кинофразы', 2),
 ('Ошибочка вышла', 2),
 ('Президенты', 2),
 ('Станки', 2),
 ('Чашка чая', 2),
 ('Бароны', 3),
 ('Вот так номер!', 3),
 ('Калмыкия', 3),
 ('Маленькие хищники', 3),
 ('На помощь!', 3),
 ('Параллели и меридианы', 3)]

In [12]:
cursor_prod.execute("""SELECT column_name 
                       FROM information_schema.columns 
                       WHERE table_name = 'themes';""")
columnsname = [_[0] for _ in cursor_prod.fetchall()]
print(columnsname)

['id', 'title', 'lap']


In [13]:
# запись данных в таблицу заказы - "themes"
cursor_prod.executemany("""INSERT INTO themes
                        (title, lap)
                        VALUES (%s, %s);""", themes)
connect_prod.commit() 

In [14]:
# проверка записи данных в таблицу "themes"
cursor_prod.execute("""SELECT *
                       FROM themes
                       LIMIT 5;""")
cursor_prod.fetchall()

[(1, 'Зоопарки', 1),
 (2, 'Книжные разбойники', 1),
 (3, 'Меткий стрелок', 1),
 (4, 'Семья композитора', 1),
 (5, 'Транспорт', 1)]

In [15]:
# получение id наименования темы
def select_theme_id(title: str) -> int:

    """ функция получения идентификатора темы
    по ее наименованию

    Args:
        theme_title (str): наименование темы

    Returns:
        int: идентификатор темы
    """

    cursor_prod.execute("""SELECT id
                    FROM themes
                    WHERE title = (%s);""", (title,))
    
    return cursor_prod.fetchone()[0]

In [12]:
select_theme_id("Станки")

5

In [16]:
# вопросы игры
cursor_dev.execute("""SELECT question_title, answer, points, image_href, audio_href, theme_title
                      FROM questions
                      INNER JOIN themes USING(theme_id);""")
questions = cursor_dev.fetchall()
questions[:2]

[('На вопрос, как им удавалось сохранить семейное счастье, эти супруги отвечали: «Зная мы ответ, давно бы получили Нобелевскую премию по психологии».',
  'Майя Плисецкая и Родион Щедрин',
  300,
  '/w/images/f/f9/RU-SI-2017-01-21-4.jpg',
  None,
  'Семья композитора'),
 ('В стихотворении «Разбойник» 1915 года он писал:\r\n«Стухнут звёзды, стухнет месяц,\r\nСтихнет песня соловья,\r\nВ чернобылье перелесиц\r\nС кистенём засяду я».',
  'Сергей Александрович Есенин',
  300,
  None,
  None,
  'Книжные разбойники')]

In [17]:
cursor_prod.execute("""SELECT column_name 
                       FROM information_schema.columns 
                       WHERE table_name = 'questions';""")
columnsname = [_[0] for _ in cursor_prod.fetchall()]
print(columnsname)

['id', 'title', 'theme_id', 'answers', 'points', 'image_href', 'audio_href', 'media_id']


In [18]:
# запись в базу данных
def insetr_questions(question_title: str, answers: str, points: int, image_href: str, audio_href: str, theme_title: str) -> None:

    theme_id = select_theme_id(theme_title)

    question_title = question_title.replace("\n", "%0a")

    cursor_prod.execute("""INSERT INTO questions
                        (title, theme_id, answers, points, image_href, audio_href)
                        VALUES(%s, %s, %s, %s, %s, %s);""", (question_title, theme_id, answers, points, image_href, audio_href,))
    connect_prod.commit()

In [20]:
for question_title, answers, points, image_href, audio_href, theme_title in questions:
    insetr_questions(question_title, answers, points, image_href, audio_href, theme_title)

In [21]:
# проверка записи данных в таблицу "questions"
cursor_prod.execute("""SELECT *
                       FROM questions
                       LIMIT 5;""")
cursor_prod.fetchall()

[(1,
  'На вопрос, как им удавалось сохранить семейное счастье, эти супруги отвечали: «Зная мы ответ, давно бы получили Нобелевскую премию по психологии».',
  4,
  'Майя Плисецкая и Родион Щедрин',
  300,
  '/w/images/f/f9/RU-SI-2017-01-21-4.jpg',
  None,
  None),
 (2,
  'В стихотворении «Разбойник» 1915 года он писал:\r%0a«Стухнут звёзды, стухнет месяц,\r%0aСтихнет песня соловья,\r%0aВ чернобылье перелесиц\r%0aС кистенём засяду я».',
  2,
  'Сергей Александрович Есенин',
  300,
  None,
  None,
  None),
 (3,
  'Его брак с Марии-Анне Келлер не принёс счастья ни одному из супругов. Детей у них не было, а партитуры использовались на папильотки и подставки для паштета.',
  4,
  'Йозеф Гайдн',
  500,
  None,
  '/w/images/c/c2/RU-SI-2017-01-21-1.mp3',
  None),
 (4,
  'Работая над «Парсифалем», он, по его словам, не мог сочинить ни ноты, если рядом не было жены Козимы. На день рождения супруги он нанял оркестр, исполнивший прелюдию к «Парсифалю».',
  4,
  'Рихард Вагнер',
  400,
  None,
  Non

In [22]:
def _build_query(host: str, method: str, params: dict) -> str:

    """функция генерации запроса в VK API

    Args:
        host (str): хост
        method (str): метод
        params (dict): параметры

    Returns:
        str: запрос
    """

    url = host + method + "?"
    if "v" not in params:
        params["v"] = "5.131"
    url += "&".join([f"{k}={v}" for k, v in params.items()])
    
    return url

In [23]:
# сервер для загрузки фото
query = _build_query(
    "https://api.vk.com/",
    "method/photos.getMessagesUploadServer",
    {"group_id": GROUP_ID,
    "access_token": TOKEN}
    )
query

'https://api.vk.com/method/photos.getMessagesUploadServer?group_id=204178645&access_token=vk1.a.U_fU3KNn13l0emj5gMgEfBmRTOWeIVG5Bu8BrubjQOSv3pTjLFyG8thGjjYHE4nA40DrL_i-JSwH8jLBz8TRzFL4v_bNhIcNzPh-n5cERJ_NKXHNm4Ikg1D3uuhxsFrXg1rtLypWEhjLtwu7Pf4MpgHOd9JGdOpSJrQs-OfCbdw5sW-uzk1hyH_L6S28kwfythPjbaG3FoUl_Gygrg4bPg&v=5.131'

In [24]:
# сервер для загрузки фото
resposen = requests.get(query)
resposen_body = resposen.json()
resposen_body

{'response': {'album_id': -64,
  'upload_url': 'https://pu.vk.com/c231331/ss2269/upload.php?act=do_add&mid=52428910&aid=-64&gid=204178645&hash=9a10a515b0f4924a4bb3aec8797eb936&rhash=f15f3330bb34f320db71c2de5f31b992&swfupload=1&api=1&mailphoto=1',
  'user_id': 0,
  'group_id': 204178645}}

In [25]:
# адрес сервера
upload_url = resposen_body["response"]["upload_url"]
upload_url

'https://pu.vk.com/c231331/ss2269/upload.php?act=do_add&mid=52428910&aid=-64&gid=204178645&hash=9a10a515b0f4924a4bb3aec8797eb936&rhash=f15f3330bb34f320db71c2de5f31b992&swfupload=1&api=1&mailphoto=1'

In [26]:
# проверка записи данных в таблицу "questions"
cursor_prod.execute("""SELECT id, image_href
                       FROM questions
                       WHERE image_href IS NOT NULL;""")
image_href = cursor_prod.fetchall()
image_href

[(1, '/w/images/f/f9/RU-SI-2017-01-21-4.jpg'),
 (10, '/w/images/7/76/RU-SI-2017-01-21-5.jpg'),
 (15, '/w/images/5/57/RU-SI-2017-01-21-9.jpg'),
 (19, '/w/images/8/8d/RU-SI-2017-01-21-10.jpg'),
 (30, '/w/images/6/60/RU-SI-2017-01-21-8.jpg'),
 (31, '/w/images/4/43/RU-SI-2017-01-21-11.jpg'),
 (35, '/w/images/b/bc/RU-SI-2017-01-21-12.jpg'),
 (41, '/w/images/7/7f/RU-SI-2017-01-21-13.jpg'),
 (43, '/w/images/2/22/RU-SI-2017-01-21-14.jpg'),
 (44, '/w/images/f/fa/RU-SI-2017-01-21-15.jpg'),
 (45, '/w/images/1/10/RU-SI-2017-01-21-18.jpg'),
 (47, '/w/images/f/fb/RU-SI-2017-01-21-19.jpg'),
 (54, '/w/images/7/73/RU-SI-2017-01-21-20.jpg'),
 (62, '/w/images/4/48/RU-SI-2017-01-21-21.jpg'),
 (69, '/w/images/e/ed/RU-SI-2017-01-21-22.jpg'),
 (73, '/w/images/5/5c/RU-SI-2017-01-21-23.jpg'),
 (90, '/w/images/9/98/RU-SI-2017-01-21-24.jpg')]

In [27]:
len(image_href)

17

In [28]:
image_href[0][1].split("/")[-1]

'RU-SI-2017-01-21-4.jpg'

In [29]:
def get_image_data(path):
    # загрузка фото вопроса
    resposen = requests.post(upload_url, files={"photo" : open(rf"C:\Users\rus_k\my_game\content\images\{path}", "rb")})
    resposen_body = resposen.json()
    # парамептры доступа
    server = resposen_body["server"]
    photo = resposen_body["photo"]
    hash = resposen_body["hash"]
    return server, photo, hash

In [30]:
def post_image(server, photo, hash, TOKEN): # загрузка фото

    query = _build_query(
        "https://api.vk.com/",
        "method/photos.saveMessagesPhoto",
        {"access_token": TOKEN,
        "photo" : photo,
        "server" : server,
        "hash" : hash
        }
        )
    # загрузка фото
    resposen = requests.get(query)
    resposen_body = resposen.json()
    #  параметры запроса
    media_id = resposen_body["response"][0]["id"]

    return media_id

In [31]:
# запись в базу данных
def update_image_questions(id: int, media_id: int) -> None:


    cursor_prod.execute("""UPDATE questions
                        SET media_id = (%s)
                        WHERE id = (%s);""",
                        (media_id, id)) 
    connect_prod.commit()


In [32]:
for id, href in image_href:
    
    path = href.split("/")[-1]

    server, photo, hash = get_image_data(path)
    media_id = post_image(server, photo, hash, TOKEN)
    print(media_id)
    update_image_questions(id, media_id)

457239096
457239097
457239098
457239099
457239100
457239101
457239102
457239103
457239104
457239105
457239106
457239107
457239108
457239109
457239110
457239111
457239112


In [ ]:
connect_prod.close()
connection_dev.close()